<a href="https://colab.research.google.com/github/Mostafa-Rawash/facebook_markiting_api/blob/main/Get_facebook_interests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
from urllib.parse import quote
import threading  
from pymongo import MongoClient

In [ ]:
# make unterest have one object for loop on it in he database is empty
all_interest_list =[{"id": "6003956183466", "name": "Listening to music", "type": "null", "path": "null", "description": "null", "source": "null", "partner": "null", "audience_size": 3839630, "country": "null", "country_access": "null", "topic": "null"}]
done_list = []


# FAceBokk URL init.

# select the type of request adinterestsuggestion => to get suggestion around ur word 
reqType = 'adinterestsuggestion' 
# this Acces Token will be Authorized for one month and to get new one will use this request:=>  "https://graph.facebook.com/v9.0/oauth/access_token?grant_type=fb_exchange_token&client_id=<APP ID>&client_secret=<APP secret code>&fb_exchange_token=<short_Access-Token>"
# short_Access-Token come from :=> "https://developers.facebook.com/tools/explorer/"
access_token = "<Enter_YOUR_ACCESS_TOKEN_HERE>"

#  Mongo DB init.

Mongo_userName= '<mongo user name>'
Mongo_password = '<mongo user password>'
client = MongoClient('<write here your mongo atlas connection link>')
db = client."<the data base name>"
db = client.get_database("<the data base name>")
# db = client.get_database("interests")
interstdb = db.'<collection name>'

In [ ]:
# will read data from database to work in it
# return 'all_interest_list' array that contain all intarsts as an object 
# retutn 'done_list' array that contain all intarsts name as a string
def get_data():
  global done_list
  global all_interest_list 
  for item in interstdb.find():
    del(item["_id"])        # remove the "_id" that generted by deafult with mongoosDB and remove it make the comper between this data and new one 
    all_interest_list.append(item) 
    print(len(all_interest_list)) 
  for item in donedb.find():
    # del(item["_id"])        # remove the "_id" that generted by deafult with mongoosDB and remove it make the comper between this data and new one 
    done_list.append(item["name"])   # item["data"]   come as array 
 

In [ ]:
# save the data in json file
def save_json_file(data , file_name):
  if len(all_interest_list) == 0 :
     get_data()
  with open(file_name, 'w') as fp:
        json.dump(data,fp)


In [ ]:
#  this function for delete repeated item in database
def cleaning_collection(collection_name):  
  collectionData = []
  x = 0
  for item in collection_name.find():
      del(item["_id"])
      if item not in collectionData:
        collectionData.append(item)
  collection_name.delete_many({})
  print(len(collectionData))
  collection_name.insert_many(collectionData)  
 

In [ ]:
#   this function for append chunk_interests & chunk_done that come generate after 1000 request  
def append(c_i , c_d):
          if len(c_i) > 0:
            interstdb.insert_many(c_i)
            c_i = []
          print(" the c_i is added")
          if len(c_d) > 0:
            donedb.insert_many(c_d)            
            c_d= []
          print(" the c_d is added")
          print("END")


In [ ]:
# getting facebook API interests using  
#  'access_token' and 'reqType' is a global var.
# interest should to be an string to convert into array 
def facebook_api(access_token, interest, reqType):
  working_list = [] 
  working_list.append(item["name"])
  interests_list_url = quote(working_list.__str__())
  URL = "https://graph.facebook.com/v9.0/search?access_token={}&interest_list={}&pretty=0&type={}&limit=100".format(access_token, interests_list_url, reqType)
  return requests.get(URL)  


In [ ]:
def main():
    chunk_intersts = []
    chunk_done = []
    count = int(0)
    for item in all_interest_list:
        chunk_intersts = []
        chunk_done = []
        if count < 20001: # to save the data evry 10000 loop 
        # ///////////////////// prolem ( the element that com after 1000 time's don't make request in it   )
            if item["name"] not in done_list: # to know in the name of interest that come from "all_interest_list" make request or  not if yes   add  count +1 
                response = facebook_api(access_token, working_list_url, reqType)
                if response.status_code == 200:
                    done_list.append(item["name"])
                    chunk_done.append({'name':item["name"]})
                    # print(len(done_list) , ' done_list')
                    data = response.json()["data"]
                    for interest in data:
                        if interest not in all_interest_list:
                            all_interest_list.append(interest)
                            chunk_intersts.append(interest)
                else:
                    print('error in response  => \n ' ,' status_code =' ,response.status_code , "   " , response , "   " , URL)
                    break
            count += 1
        else:
          print("start")
          print(len(done_list) ,"   from   " , len(all_interest_list))
          threading.Thread(target=append,args=(chunk_intersts,chunk_done,)).start()
          count = 0
    print(len(done_list) ,"   from   " , len(all_interest_list))
    print("start")
    threading.Thread(target=append,args=(chunk_intersts,chunk_done,)).start()
 

In [ ]:
get_data()
print("data is ready")


In [ ]:
main()

In [ ]:
save_json_file(all_interest_list , '/content/drive/MyDrive/ensoulify/Facebook Bert/facefook_interests.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 
data = pd.DataFrame.from_dict( all_interest_list )
data = data.drop_duplicates(subset=['id', 'name'], keep='last')  
save_json_file(data.to_dict() , '/content/drive/MyDrive/ensoulify/Facebook Bert/facebook_interests.json')
